# Using Python as a Replacement for Shell
## Bryan Raney, Rutgers University

Tasks typically done with the shell:
* Interact with the Shell
* Explore Files and Directories
* Manipulate Files and Directories
* Run Programs
* View and Manipulate Environment Variables

### Interact with the Shell
IPython provides filename tab-completion and limited command history.

IPython provides these magic commands:
* `!` for running commands directly in the shell
* `%env` to handle environment variables
* `%cd` to change directories
* `%mkdir` to make a directory
* etc.

The magic commands don't take variables as inputs, so are less useful for programmatic tasks.

In [ ]:
# Using Python 3.5
import os
import os.path
import stat
import shutil
import glob
import subprocess
os.chdir(os.path.join(os.environ['HOME'],'Documents','Work','AMS_2017','Python_for_Shell'))

### Explore Files and Directories

List files in the current directory

In [ ]:
files = !ls -A
print(files)

In [ ]:
files = os.listdir()
print(files)

Determine what the working directory is

In [ ]:
!pwd

In [ ]:
%pwd

In [ ]:
cwd = os.getcwd()
print(cwd)

Does a certain directory exist?

In [ ]:
datapath = os.path.join('tutorial', 'data') # build a path (relative to cwd)
print('datapath:',datapath)
print('exists:', os.path.exists(datapath))

Is it really a directory?

In [ ]:
os.path.isdir(datapath)

Change working directory to something else

In [ ]:
!pwd
print('--------')
!cd tutorial ; pwd
print('--------')
!pwd

**WARNING:** Directory change with `!` does not persist!

In [ ]:
!pwd
print('--------')
%cd tutorial

In [ ]:
%cd(datapath)

In [ ]:
%cd ..

In [ ]:
print(os.getcwd())
print('--------')
os.chdir(datapath)
print(os.getcwd())
os.chdir('..')
print(os.getcwd())

Obtain details of a file (e.g. permissions, ownership, size, date)

In [ ]:
!ls -l somefile.txt

In [ ]:
filename = 'somefile.txt'
stat_info = os.stat(filename)
print('stat_info:', stat_info)

import pwd # for Unix-like "passwd" data
owner = pwd.getpwuid(stat_info.st_uid).pw_name
print('owner:', owner)

file_mode = stat.S_IMODE(stat_info.st_mode)
print('mode:', '%o' % file_mode)

import datetime
last_modified = datetime.datetime.fromtimestamp(stat_info.st_mtime)
print('last modified:', last_modified)

### Manipulate Files and Directories

Change permissions of a file

In [ ]:
!ls -l somefile.txt
print('--------')
os.chmod(filename, file_mode | stat.S_IRGRP) # modifying previous mode
!ls -l somefile.txt

Change group owner of a file

In [ ]:
!ls -l somefile.txt
print('--------')

group_name = 'everyone'

uid = -1 # means keep same
import grp # for Unix-like "group" data
gid = grp.getgrnam(group_name).gr_gid
os.chown(filename, uid, gid)
!ls -l somefile.txt

Create a new directory

In [ ]:
dir_name = 'bbbbbb'

!ls
print('----------')

%mkdir aaaaaa
os.mkdir(dir_name)

!ls

Remove an empty directory

In [ ]:
%rmdir aaaaaa
os.rmdir(dir_name)

!ls

Copy a file to a new location

In [ ]:
!ls
print('--------')
%cp somefile.txt somefile2.txt
shutil.copy2('somefile.txt', 'somefile3.txt')
!ls

Rename a file

In [ ]:
!ls
print('--------')
%mv somefile2.txt foo.txt
shutil.move('somefile3.txt', 'bar.txt')
!ls

Delete a file

In [ ]:
!ls
print('--------')

%rm foo.txt
os.remove('bar.txt') # !! os, not shutil

!ls

Delete all files that match some criteria (e.g. `*~`)

In [ ]:
!ls testdir

In [ ]:
for file in glob.glob('testdir/*~'):
    os.remove(file)
!ls testdir

Delete a non-empty directory (recursively)

In [ ]:
!ls
print('--------')
%rmdir testdir
!ls

In [ ]:
!ls
print('--------')
shutil.rmtree('testdir')
!ls

Create a symbolic link to a directory

In [ ]:
!ln -s ~/stuff foo
!ls

In [ ]:
os.symlink(os.path.join(os.environ['HOME'], 'stuff'), 'foo2')
!ls

In [ ]:
!ls -l

### Run Programs

Create an empty file with 'touch'

In [ ]:
!ls
print('--------')
subprocess.run(['touch', 'blank.txt'], check=True)
!ls

Concatenate some netcdf files with 'ncrcat'

In [ ]:
%cd data

In [ ]:
!ls -1 *air*
infiles = glob.glob('air.sig995.????.nc')
outfile = 'all_air.nc'
command = ['ncrcat'] + infiles + [outfile]
print('command:', command)
#os.remove(outfile)
subprocess.run(command, check=True)
print('--------')
!ls -1 *air*

Chain commands using pipes

Find all the 'units' attributes in a NetCDF file using 'ncdump' and 'grep'

In [ ]:
!ncdump -h all_air.nc | grep -i 'units'

In [ ]:
ncdump = subprocess.Popen(['ncdump', '-h', outfile], stdout=subprocess.PIPE)
grep = subprocess.Popen(['grep', '-i', 'units'], stdin=ncdump.stdout, stdout=subprocess.PIPE)
ncdump.stdout.close()  # Allow ncdump to receive a SIGPIPE if grep exits.
output = grep.communicate()[0].decode('utf8')
print(output)

Run a command in the background

In [ ]:
sleep = subprocess.Popen(['sleep', '5'])
print('before')
!sleep 1
print('after 1')
sleep.wait()
print('after 5')

### View and Maniuplate Environment Variables

In [ ]:
%env USER

In [ ]:
os.environ['USER']

In [ ]:
%env TUTORIAL

In [ ]:
os.environ['TUTORIAL']

In [ ]:
%env TUTORIAL = Shell Replacement

In [ ]:
%env TUTORIAL

In [ ]:
os.environ['TUTORIAL'] = 'Replacing the Shell'

In [ ]:
os.environ['TUTORIAL']